In [18]:
# same imports as above but saves me time from scrolling up when restarting the notebook.
from dask.distributed import Client, LocalCluster, Nanny, SchedulerPlugin, WorkerPlugin, Future, secede, rejoin
from dask.distributed import print as dprint
from dask.highlevelgraph import HighLevelGraph
from dask.distributed import Queue, Variable, Event, get_client, worker_client
import numpy as np
import dask
import dask.array as da
import dask.dataframe as dd
import graphviz
import logging
import itertools
import time
import zmq

In [19]:
np.random.seed(42)

In [20]:
client = Client(scheduler_file = "scheduler.json")
q0 = Queue("rank0")
q1 = Queue("rank1")

In [21]:
np_arr = []
for i in range(5):
    data0 = np.random.rand(1,5,20)
    data1 = np.random.rand(1,5,20)
    #data0 = np.zeros((1,5,20))
    #data1 = np.ones((1,5,20))
    np_arr.append((data0,data1))
    # w0 gets zeros
    f0 = client.scatter(data0, direct = True, workers = ["worker0"])
    # w1 gets ones
    f1 = client.scatter(data1, direct = True, workers = ["worker1"])

    q0.put(f0)
    q1.put(f1)
    # to mimic process taking a while
    time.sleep(10)


In [22]:
np_full = [np.concatenate(pair,axis=1) for pair in np_arr]
np_full = np.vstack(np_full)

In [23]:
def Derivee(F, dx):
    """
    First Derivative
    Input: F = function to be derived
           dx = step of the variable for derivative
    Output: dFdx = first derivative of F
    """
    c0 = 2.0 / 3.0
    dFdx = c0 / dx * (F[3:-1] - F[1:-3] - (F[4:] - F[:-4]) / 8.0)
    return dFdx

In [24]:
print(Derivee(np_full, 1).mean())

-0.01535692703889472
